In [ ]:
from jetcam.usb_camera import USBCamera
from jetcam.csi_camera import CSICamera
import torchvision.transforms as transforms
from dataset import ImageClassificationDataset
import ipywidgets
import traitlets
from IPython.display import display
from jetcam.utils import bgr8_to_jpeg
import ipywidgets
import traitlets
from IPython.display import display
from jetcam.utils import bgr8_to_jpeg

In [ ]:
# Camera
camera = CSICamera(width=224, height=224, capture_device=0)
camera.running = True
print("camera created")

In [ ]:
# Categories and Dataset
CATEGORIES = ['background', 'maya', 'neg_basil', 'neg_coco', 'pos_chain', 'pos_cluster']
DATASETS = ['gram_stain']
TRANSFORMS = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])
datasets = {}
for name in DATASETS:
    datasets[name] = ImageClassificationDataset(name, CATEGORIES, TRANSFORMS)
print("{} categories defined".format(CATEGORIES))

In [ ]:
# Data Collection Widget
dataset = datasets[DATASETS[0]]
camera.unobserve_all()
camera_widget = ipywidgets.Image()
traitlets.dlink((camera, 'value'), (camera_widget, 'value'), transform=bgr8_to_jpeg)

dataset_widget = ipywidgets.Dropdown(options=DATASETS, description='dataset')
category_widget = ipywidgets.Dropdown(options=dataset.categories, description='category')
count_widget = ipywidgets.IntText(description='count')
save_widget = ipywidgets.Button(description='add')
count_widget.value = dataset.get_count(category_widget.value)

def set_dataset(change):
    global dataset
    dataset = datasets[change['new']]
    count_widget.value = dataset.get_count(category_widget.value)
dataset_widget.observe(set_dataset, names='value')

def update_counts(change):
    count_widget.value = dataset.get_count(change['new'])
category_widget.observe(update_counts, names='value')

def save(c):
    dataset.save_entry(camera.value, category_widget.value)
    count_widget.value = dataset.get_count(category_widget.value)
save_widget.on_click(save)

data_collection_widget = ipywidgets.VBox([
    ipywidgets.HBox([camera_widget]), dataset_widget, category_widget, count_widget, save_widget
])

print("data_collection_widget created")

In [ ]:
display(data_collection_widget)

In [ ]:
import os
import IPython
if type(camera) is CSICamera:
    print("Ignore 'Exception in thread' tracebacks\n")
    camera.cap.release()
os._exit(0)